# Commands

In [ ]:
# titanv 1:
# screen -S asa2-train1
# source ~/.bash_profile ; source ~/.bash_aliases ; cd ~/asa2-train-logs
# DOCKER_PORT=9961 nn -o nohup-asa2-train-$HOSTNAME-1.out ~/docker/keras/run-jupython.sh ~/notebooks/asa/train/asa2-train.ipynb titanv

In [ ]:
# titanv 2:
# screen -S asa2-train2
# source ~/.bash_profile ; source ~/.bash_aliases ; cd ~/asa2-train-logs
# DOCKER_PORT=9962 nn -o nohup-asa2-train-$HOSTNAME-2.out ~/docker/keras/run-jupython.sh ~/notebooks/asa/train/asa2-train.ipynb titanv

# Inits

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
TEST = False
if isNotebook:
    TEST = False

# Force CUDA_VISIBLE_DEVICES

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
hasGPU = True

# Imports

In [ ]:
from newssource.asattribution.utils import *
from newssource.asattribution.asamin import *
from newssource.asa.asapreproc import *
from newssource.metrics.ndcg import *

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
if not isNotebook:
    matplotlib.use('Agg')
import random
import time
import pickle
from hashlib import md5

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
from random import random
from numpy import array
from numpy import cumsum

In [ ]:
import statistics

In [ ]:
from machinelearning.baseline import *
from machinelearning.encoder import *
from machinelearning.kerasutils import *
from machinelearning.kerasmodels import *
from machinelearning.iterator import *
from machinelearning.metrics import *
from machinelearning.attmap.builder import *

In [ ]:
from keras.layers import LSTM, GRU, Dense, CuDNNLSTM, CuDNNGRU, Bidirectional
from keras.layers import BatchNormalization, Activation, SpatialDropout1D, InputSpec
from keras.layers import MaxPooling1D, TimeDistributed, Flatten, concatenate, Conv1D
from keras.utils import multi_gpu_model, plot_model
from keras.layers import concatenate, Input, Dropout
from keras.models import Model, load_model, Sequential
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.callbacks import Callback, History, ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras import callbacks
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras import backend as K

# Config

In [ ]:
config = \
{
    "patience": 3 if TEST else 20, # <IMPORTANT>
    # 3gramsFiltered, 2gramsFiltered, 1gramsFiltered, textSentences # <IMPORTANT>
    # filtered_sentences, sentences
    "dataCol": "filtered_sentences", # <IMPORTANT>
    "wordVectorsPattern": "test" if TEST else "glove-840B", # <IMPORTANT>
    "embeddingsDimension": 100 if TEST else 300,
    "minVocDF": 2, # 10 if isNotebook else 2,
    "minVocLF": 2,
    "minTokensLength": 3,
    "docLength": 1200, # The size of documents representation (median * 3 = 600)
    "isTrainableEmbeddings": False,
    "doMultiGPU": False, # not isNotebook
    "batchSize": 32 if TEST else 128, # 128, 256 # <IMPORTANT>
    "epochs": 10 if TEST else 1000,
    "maxQueueSize": 10 if TEST else 10,
    "saveFinalModel": False,
    "doNotif": True, # not isNotebook,
    "attention": False, # <IMPORTANT>
    "bidirectional": False, # <IMPORTANT>
    "isCuDNN": True, # <IMPORTANT>
    "saveMetrics":
    {
        "val_loss": "min",
        "val_acc": "max",
        "val_top_k_categorical_accuracy": "max",
    },
    "metrics": ['accuracy', 'top_k_categorical_accuracy'], # ['sparse_categorical_accuracy', 'sparse_top_k_categorical_accuracy'], ['accuracy', 'top_k_categorical_accuracy']
    "loss": 'categorical_crossentropy', # sparse_categorical_crossentropy, categorical_crossentropy
    "trainStepDivider": 80 if TEST else 0, # 5 à la base......
    "infiniteBatcherShuffle": 0 if TEST else 0,
    
    "inputEncoding": "embedding", # index, embedding
    "labelEncoding": 'onehot', # onehot, index
    
    "denseUnits": [500, 100], # [100], [500, 100]
    "rnnUnits": 500,
    
    "persist": [False, False] if TEST else [False, False],
}
if config["isCuDNN"]:
    del config["isCuDNN"]

In [ ]:
# config["hostname"] = getHostname()
print("hostname: " + str(getHostname()))

In [ ]:
outputDirRoot = nosaveDir() + "/asa2-train"
print("outputDirRoot: " + outputDirRoot)

In [ ]:
config["trainPattern"] = "train" # <IMPORTANT>
config["validationPattern"] = "validation" # <IMPORTANT>

In [ ]:
dataDirectory = dataDir() + "/Asa2/kset"

In [ ]:
config["outputDir"] = outputDirRoot + "/" + objectToHash(config)
mkdir(config["outputDir"])

In [ ]:
logger = Logger(config["outputDir"] + "/asa2-train.log")

In [ ]:
tt = TicToc(logger=logger)
tt.tic()

In [ ]:
toJsonFile(toMongoStorable(config), config["outputDir"] + "/config.json")

In [ ]:
log(lts(config), logger)

# Loading of word embeddings

In [ ]:
emb = Embeddings(config["wordVectorsPattern"], config["embeddingsDimension"], verbose=True, logger=logger)

In [ ]:
wordEmbeddings = emb.getVectors()

In [ ]:
# We get the embedings dimension:
embeddingsDimension = len(wordEmbeddings["the"])

In [ ]:
tt.tic("We loaded word embeddings")

# We get files

In [ ]:
trainFilesPath = sortedGlob(dataDirectory + "/" + config["trainPattern"] + "/*.bz2")
validationFilesPath = sortedGlob(dataDirectory + "/" + config["validationPattern"] + "/*.bz2")
assert len(trainFilesPath) > 0
log("trainFilesPath:\n" + reducedLTS(trainFilesPath, 4), logger)
log("validationFilesPath:\n" + reducedLTS(validationFilesPath, 4), logger)

# We search an amount of batches to skip

In [ ]:
trainInfiniteBatcherSkip = 0
# In case we reume a previous train:
if len(sortedGlob(config["outputDir"] + "/models/ep*")) > 0:
    lastEpochPath = sortedGlob(config["outputDir"] + "/models/ep*")[-1]
    log("We found an epoch to resume: " + lastEpochPath, logger)
    batchesPassedFile = lastEpochPath + "/batchesPassed.txt"
    if isFile(batchesPassedFile):
        trainInfiniteBatcherSkip = int(fileToStr(batchesPassedFile))
        logWarning("We will skip " + str(trainInfiniteBatcherSkip) + " batches because we resume a previous train", logger)

# We prepare data

In [ ]:
prebuilt = None
prebuiltPath = config["outputDir"] + "/asap-prebuilt.pickle"

In [ ]:
if isFile(prebuiltPath):
    prebuilt = prebuiltPath
    log("We found " + prebuilt, logger)
else:
    log("We didn't found any asap prebuilt pickle file...", logger)

In [ ]:
# We init AsaPreproc:
asap = buildASAP\
(
    trainFilesPath,
    validationFilesPath,
    config["dataCol"],
    
    minTokensLength=config["minTokensLength"],

    batchSize=config["batchSize"],
    minVocDF=config["minVocDF"],
    minVocLF=config["minVocLF"],

    wordEmbeddings=wordEmbeddings,
    
    persist=config["persist"],
    
    docLength=config["docLength"],
    
    prebuilt=prebuilt,
    
    logger=logger,
    verbose=True,
    
    labelEncoding=config["labelEncoding"],
    encoding=config["inputEncoding"],
)

In [ ]:
# We serialize all:
asap.serializePrebuilt(prebuiltPath)

In [ ]:
tt.tic("All train and validation data are ready")

# We define the model

In [ ]:
opt = optimizers.Adam(clipnorm=1.0)

In [ ]:
initialEpoch = 0
if len(sortedGlob(config["outputDir"] + "/models/ep*")) > 0:
    logWarning("#" * 20 + " We will resume a previous train " + "#" * 20, logger)
    lastEpochPath = sortedGlob(config["outputDir"] + "/models/ep*")[-1]
    initialEpoch = getFirstNumber(decomposePath(lastEpochPath)[1]) + 1
    assert not isFile(config["outputDir"] + "/finished")
    # To load the model we first build it:
    modelKwargs = fromJsonFile(lastEpochPath + "/kwargs.json")
    assert modelKwargs["docLength"] == asap.getDocLength()
    assert modelKwargs["vocSize"] == len(asap.getVocIndex())
    assert modelKwargs["nbClasses"] == len(asap.getLabelEncoder())
    if "embeddingsDimension" not in modelKwargs:
        modelKwargs["embeddingsDimension"] = asap.getEmbeddingsDimension() if config["inputEncoding"] == "embedding" else None
    if "embeddingMatrix" not in modelKwargs:
        modelKwargs["embeddingMatrix"] = asap.getEmbeddingMatrix() if config["inputEncoding"] == "index" else None
    (originalModel, modelKwargs, modelScript) = buildRNN\
    (
        logger=logger,
        verbose=True,
        **modelKwargs,
    )
    # Then we load weights:
    if isFile(lastEpochPath + "/model.h5"):
        originalModel = load_model(lastEpochPath + "/model.h5")
    else:
        originalModel.load_weights(lastEpochPath + "/weights.h5") # WARNING we loose optimizer states
        # Finally we compile it:
        originalModel.compile(loss=config["loss"], optimizer=opt, metrics=config["metrics"])
    parallelModel = None
    model = originalModel
    tt.tic("We loaded the model to resume training. Initial epoch: " + str(initialEpoch), logger)
else:
    (originalModel, modelKwargs, modelScript) = buildRNN\
    (
        docLength=asap.getDocLength(),
        vocSize=len(asap.getVocIndex()),
        nbClasses=len(asap.getLabelEncoder()),
        isEmbeddingsTrainable=config["isTrainableEmbeddings"],
        
        denseUnits=config["denseUnits"],
        rnnUnits=config["rnnUnits"],
        
        embSpacialDropout=0.2,
        firstDropout=0.2,
        recurrentDropout=0.2,
        attentionDropout=0.2,
        denseDropout=0.2,
        useRNNDropout=True,
        
        isBidirectional=config["bidirectional"],
        
        isCuDNN='isCuDNN' not in config or config["isCuDNN"],
        
        rnnType='LSTM',
        addAttention=config["attention"],
        
        bnAfterEmbedding=False,
        bnAfterRNN=False,
        bnAfterAttention=False,
        bnAfterDenses=False,
        bnAfterLast=False,
        bnBeforeActivation=True,

        embeddingMatrix=asap.getEmbeddingMatrix() if config["inputEncoding"] == "index" else None,
        logger=logger,
        verbose=True,
        
        embeddingsDimension=asap.getEmbeddingsDimension() if config["inputEncoding"] == "embedding" else None,
    )
    parallelModel = None
    model = originalModel
    model.compile(loss=config["loss"], optimizer=opt, metrics=config["metrics"])

In [ ]:
model.summary()

In [ ]:
strToFile(originalModel.to_json(), config["outputDir"] + "/model.json")

# We define metrics callbacks

In [ ]:
mainCallback = KerasCallback\
(
    originalModel,
    logger=logger,
    graphDir=config["outputDir"] + "/graphs",
    modelsDir=config["outputDir"] + "/models",
    doNotif=False,
    saveMetrics=config["saveMetrics"],
    doPltShow=isNotebook,
    historyFile=config["outputDir"] + "/history.json",
    initialEpoch=initialEpoch,
    stopFile=config["outputDir"] + "/stop",
    earlyStopMonitor=\
    {
        'val_loss': {'patience': config["patience"]},
        'val_acc': {'patience': config["patience"]},
        'val_top_k_categorical_accuracy': {'patience': config["patience"]},
    },
    batchesPassed=trainInfiniteBatcherSkip,
    batchesAmount=asap.getBatchesCount(0),
    
    saveFunct=saveModel,
    saveFunctKwargs=\
    {
        "makeSubDir": False, "kwargs": modelKwargs, "script": modelScript,
        "extraInfos":\
        {
            "wordVectorsPattern": config["wordVectorsPattern"],
            "embeddingsDimension": config["embeddingsDimension"],
            "minVocDF": config["minVocDF"],
            "minVocLF": config["minVocLF"],
            "dataCol": config["dataCol"],
            "labelEncoding": config["labelEncoding"],
            "inputEncoding": config["inputEncoding"],
            "trainStepDivider": config["trainStepDivider"],
            "infiniteBatcherShuffle": config["infiniteBatcherShuffle"],
        },
    },
)

# We train the model

In [ ]:
stepsPerEpoch = asap.getBatchesCount(0)
if dictContains(config, "trainStepDivider") and config["trainStepDivider"] > 1:
    stepsPerEpoch = math.ceil(stepsPerEpoch / config["trainStepDivider"])
    log("The stepsPerEpoch was " + str(asap.getBatchesCount(0)) + " but now is " + str(stepsPerEpoch), logger)

In [ ]:
log("We launch fit_generator", logger)
asap.verbose = True # if TEST else False
history = model.fit_generator\
(
    asap.getInfiniteBatcher(0, shuffle=config["infiniteBatcherShuffle"], skip=trainInfiniteBatcherSkip),
    steps_per_epoch=stepsPerEpoch,
    validation_data=asap.getInfiniteBatcher(1),
    validation_steps=asap.getBatchesCount(1) / 40 if TEST else asap.getBatchesCount(1),
    epochs=config["epochs"],
    verbose=1,
    max_queue_size=config["maxQueueSize"],
    callbacks=[mainCallback, callbacks.TerminateOnNaN()],
    initial_epoch=initialEpoch,
)

In [ ]:
mainCallback.logHistory()
log("Best val_loss: " + str(min(mainCallback.history["val_loss"])), logger)
log("Best val_acc: " + str(max(mainCallback.history["val_acc"])), logger)
log("Best val_top_k_acc: " + str(max(mainCallback.history["val_top_k_categorical_accuracy"])), logger)
log("Nb epochs: " + str(len(mainCallback.history["val_loss"])), logger)

In [ ]:
if not isNotebook:
    notif("Training done", lts(config))

# We save the model and infos

In [ ]:
try:
    # toJsonFile(history.history, config["outputDir"] + "/history.json")
    toJsonFile({"epochs": mainCallback.epochs, "history": mainCallback.history},
               config["outputDir"] + "/history.json")
except Exception as e:
    logException(e, logger)

In [ ]:
if config["saveFinalModel"]:
    finalModelDirectory = config["outputDir"] + "/final-model"
    mkdir(finalModelDirectory)
    originalModel.save(finalModelDirectory + '/model.h5')

In [ ]:
touch(config["outputDir"] + "/finished")

# Attention

In [ ]:
if isNotebook:
    asap.setParallelProcesses(1) # For consistency

In [ ]:
if isNotebook:
    # We get val encoded tokens:
    valEncodedTokens = np.array([tokens for tokens, encodedAd in asap.getPart(1)])
    # We get documents non-lowered and non-flattened, with masks (pre-padding):
    valPaddedDocs = [tokens for tokens, _ in asap.getRawPart(1, pad=True)]
    # We get all encoded labels:
    valEncodedLabels = np.array([encodedAd for tokens, encodedAd in asap.getPart(1)])
    # We get all labels:
    valLabels = [ad for tokens, ad in asap.getRawPart(1)]
    # We get the dict label -> encodedLabel (we can also use asap.encodedLabelToLabel(encodedLabel)):
    encodedAds = asap.getLabelEncoder()
    # We display it:
    bp(valPaddedDocs)

In [ ]:
if isNotebook:
    # We get predictions:
    predictionsAsSoftmax = model.predict(valEncodedTokens)
    # We convert it to encoded labels according to the max probability of softmax vectors:
    predictionsAsEncodedLabel = []
    for i in range(len(predictionsAsSoftmax)):
        predSoftmax = predictionsAsSoftmax[i]
        predEncodedLabel = np.zeros(len(predSoftmax))
        predEncodedLabel[np.argmax(predSoftmax)] = 1
        predictionsAsEncodedLabel.append(predEncodedLabel)
    # And we convert all to labels:
    predictionsAsLabel = [asap.decodeLabel(enc) for enc in predictionsAsEncodedLabel]
    # We display it:
    bp(predictionsAsLabel)

In [ ]:
if isNotebook:
    # We get attentions:
    attentions = getAttentions(model, valEncodedTokens)

In [ ]:
if isNotebook:
    # Now we compute the accuracy:
    wellClassifiedCount = 0
    for i in range(len(predictionsAsLabel)):
        if valLabels[i] == predictionsAsLabel[i]:
            wellClassifiedCount += 1
    print("Accuracy: " + str(truncateFloat(wellClassifiedCount / len(predictionsAsLabel) * 100.0, 2)))

In [ ]:
if isNotebook:
    # And finally we print some attentions:
    for i in range(10, 50):
        doc = valPaddedDocs[i]
        label = valLabels[i]
        predLabel = predictionsAsLabel[i]
        okToken = "==> OK <==" if label == predLabel else "==> FAIL <=="
        print(okToken + " Prediction: " + label[:30] + ", Ground truth: " + str(predLabel)[:30])
        attention = attentions[i]
        showAttentionMap(doc, attention)

# End

In [ ]:
if config["doNotif"]:
    notif("LARGE training done on " + getHostname())

In [ ]:
tt.toc()